# ML AB Testing

## Imports

In [1]:
import scipy
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
# from ABTesting import DfHelper

In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from df_helper import DfHelper

## Data reading

In [4]:
helper = DfHelper()
missing_values = ["n/a", "na", "undefined"]
df = helper.read_csv("../data/AdSmartABdata.csv")
df.head(5)

2021-07-24 14:47:44,151 — DfHelper — DEBUG — file read as csv


,auction_id,experiment,date,hour,device_make,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,0,0


## Linear

In [5]:
from config import Config

In [8]:
X_train = pd.read_csv(str(Config.FEATURES_PATH / "train_features.csv"))
y_train = pd.read_csv(str(Config.FEATURES_PATH / "train_labels.csv"))


In [9]:
X_train

,experiment,hour,date_of_week,date_of_month,month,device_make
0,1,20,2,8,7,45
1,1,7,5,4,7,45
2,1,11,4,10,7,177
3,0,13,4,10,7,45
4,1,1,4,10,7,45
...,...,...,...,...,...,...
7264,0,15,4,3,7,45
7265,0,15,3,9,7,45
7266,1,6,0,6,7,177
7267,0,6,4,10,7,45


In [12]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
kf = KFold(n_splits = 5)
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
avg_score = 0
best_solver = solvers[0]

for solver in solvers:
    scores = []
    model = LogisticRegression(solver=solver, random_state=42)
    randomIter = kf.split(X_train)
    for i in range(5):
        train_index, val_index = next(randomIter)
        _X_train = X_train.iloc[train_index]
        _y_train = y_train.iloc[train_index]

        _X_val = X_train.iloc[val_index]
        _y_val = y_train.iloc[val_index]

        model.fit(X_train, y_train.to_numpy().ravel())
        y_preds = model.predict(_X_val)
        score = accuracy_score(_y_val, y_preds)
        scores.append(score)

    avg_score_for_solver = sum(scores)/len(scores)
    if(avg_score_for_solver > avg_score):
        avg_score = avg_score_for_solver
        best_solver = solver

In [13]:
def train_model(_model):
  kf = KFold(n_splits = 5)
  solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
  avg_score = 0
  best_model = None
  best_solver = solvers[0]

  for solver in solvers:
      scores = []
      model = _model(solver)
      randomIter = kf.split(X_train)
      for i in range(5):
          train_index, val_index = next(randomIter)
          _X_train = X_train.iloc[train_index]
          _y_train = y_train.iloc[train_index]

          _X_val = X_train.iloc[val_index]
          _y_val = y_train.iloc[val_index]

          model.fit(X_train, y_train.to_numpy().ravel())
          y_preds = model.predict(_X_val)
          score = accuracy_score(_y_val, y_preds)
          scores.append(score)

      avg_score_for_solver = sum(scores)/len(scores)
      if(avg_score_for_solver > avg_score):
          avg_score = avg_score_for_solver
          best_solver = solver
          best_model = model

  # return the best model
  return best_model, best_solver

In [14]:
def model(solver):
    model = LogisticRegression(solver=solver, random_state=42)
    return model

In [14]:
train_model(model)


,aware
0,0
1,0
2,0
3,0
4,0
...,...
6456,0
6457,0
6458,0
6459,0


In [1]:
import xgboost


In [ ]:
xgboost??